# Solubility of uranium for changing pH

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Feb 4th, 2022</p>

This tutorial presents an example of the calculation of the uranium solubility for changing pH levels in underground
water. In this example, the thermodynamic database `psinagra-12-07` is used based on
[Nagra/PSI Chemical Thermodynamic Data Base 01/01](https://www.psi.ch/de/les/database). It supports the ongoing safety
assessments for the planned repositories for low- and intermediate-level (L/ILW) and high-level (HLW) radioactive waste
in Switzerland.

```{note}

This tutorial is joint work with Dan Miron, PSI.

If your main interest is in computing thermodynamic properties, rather than chemical equilibrium and
kinetics modeling, you may want to check [ThermoFun](https://thermohub.org/thermofun/thermofun/),
which is an excellent project dedicated to this task.
```

We set up the chemical system with the aqueous phase only.

In [ ]:
from reaktoro import *
import numpy as np
import math

# Define Thermofun database
db = ThermoFunDatabase ("psinagra-12-07")

# Define aqueous phase
solution = AqueousPhase(speciate("Al C Ca Cl Fe H K Mg N Na O P S Si U"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

# Define chemical system by providing database, aqueous phase, and minerals
system = ChemicalSystem(db, solution)

The specified below equilibrium specs indicate that equilibrium calculations will be performed for fixed T, P, and pH.
The corresponding equilibrium conditions and equilibrium solver are initialized using `specs` instance:

In [ ]:
# Specify conditions to be satisfied at chemical equilibrium
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()

# Define temperature and pressure
T = 25.0 # in Celsius
P = 1.0 # in bar

# Define conditions to be satisfied at chemical equilibrium
conditions = EquilibriumConditions(specs)
conditions.temperature(T, "celsius")
conditions.pressure(P, "bar")

# Define equilibrium solver
solver = EquilibriumSolver(specs)

The initial chemical state is defined according to the following recipe:

* 1000 g of water,
* 1e-5 mol of H<sub>3</sub>PO<sub>4</sub>@,
* 1e-5 mol of UO<sub>2</sub>(SO<sub>4</sub>)@, and
* 0.1 g of CO<sub>2</sub>@.

In [ ]:
# Define initial equilibrium state
state = ChemicalState(system)
state.set("H2O@"     , 1e3,  "g")
state.set("CO2@"     , 1e-1, "g")
state.set("H3PO4@"   , 1e-5, "mol")
state.set("UO2(SO4)@", 1e-5, "mol")

The amount of uranium can be calculated through the chemical properties of this chemical state.
We define the range of the pH values as well as the list of uranium-containing species interesting for our modeling.
Finally, we run simulations in the for-loop over the pH values that are provided to the equilibrium conditions instance:

In [ ]:
# Calculate the amount of uranium element
prop = ChemicalProps(state)
bU = prop.elementAmount("U")[0]

# Defined auxiliary arrays
pHs = np.linspace(5, 10, num=41)
species_list = SpeciesList("UO2+2 UO2OH+ UO2(OH)2@ UO2CO3@ (UO2)3CO3(OH)3+ (UO2)2(OH)+3 "
                               "UO2(CO3)3-4 UO2(CO3)2-2 (UO2)2(OH)2+2 (UO2)3(OH)5+ (UO2)4(OH)7+")
nU = np.zeros((len(pHs), species_list.size()))

for i in range(0, len(pHs)):

    # Set the value of pH for the current equilibrium calculations
    conditions.pH(pHs[i])

    # Equilibrate the initial state with given conditions and component amounts
    res = solver.solve(state, conditions)

    if not res.optima.succeeded:
        # If the equilibrium calculations has not succeeded, return nan values
        nU[i, :] = math.nan * np.ones(species_list.size())
    else:
        # Otherwise, calculate U(VI) Speciation, %
        for j in range(0, species_list.size()):#species in species_list:
            nU[i, j] = state.speciesAmount(species_list[j].name())[0] / bU * 100


Below, we plot obtained values of the U(VI) Speciation in % on two plots. First, we plot species having a higher range
of values.

In [ ]:
import matplotlib.pyplot as plt
colors = ['C1', 'C2', 'C3', 'C4', 'C5', 'C7', 'C8', 'C9', 'C0', 'darkblue', 'darkgreen']

#"UO2+2 UO2OH+ UO2CO3@ UO2(CO3) 3-4 UO2(CO3)2-2"
indices_high = [0, 1, 3, 6, 7]

#"UO2(OH)2@ (UO2)3CO3(OH)3+ (UO2)2(OH)+3 (UO2)2(OH)2+2 (UO2)3(OH)5+ (UO2)4(OH)7+"
indices_low = [2, 4, 5, 8, 9, 10]

plt.figure()
plt.xlabel("pH")
plt.ylabel("U(VI) Speciation, %")
for j in indices_high:
    plt.plot(pHs, nU[:, j], label=species_list[j].name(), color=colors[j])
plt.legend(loc="best")
plt.grid()

Next, we also depict the U(VI) speciation of species with lower values.

In [ ]:
plt.figure()
plt.xlabel("pH")
plt.ylabel("U(VI) Speciation, %")
for j in indices_low:
    plt.plot(pHs, nU[:, j], label=species_list[j].name(), color=colors[j])
plt.legend(loc="best")
plt.grid()